In [4]:
import sys
print(sys.executable)
print(sys.version)

!pip install torch 

!pip install transformers

c:\Users\igorm\AppData\Local\Programs\Python\Python311\python.exe
3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]


In [5]:
import json
from sklearn.model_selection import train_test_split

with open('output.json', encoding='utf-8') as f:
    data = json.load(f)

inputs = [item['input'] for item in data]
outputs = [
    f"code: {item['output']['catalog_catalog.code']} description: {item['output']['catalog_catalog.description']} "
    f"full_description: {item['output']['catalog_catalog.full_description']} similar_parts: {item['output']['catalog_catalog.similar_parts']} "
    f"application_table: {item['output']['catalog_catalog.application_table']} brand_producer: {item['output']['catalog_catalog.brand_producer']}"
    for item in data
]


In [6]:
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)


In [7]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

model_name = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

class T5Dataset(Dataset):
    def __init__(self, inputs, outputs, tokenizer, max_length=512):
        self.inputs = inputs
        self.outputs = outputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        # Tokenização
        input_encoding = self.tokenizer(
            input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        output_encoding = self.tokenizer(
            output_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        labels = output_encoding['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
    
train_dataset = T5Dataset(train_inputs, train_outputs, tokenizer)
test_dataset = T5Dataset(test_inputs, test_outputs, tokenizer)

c:\Users\igorm\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
import tempfile


temp_dir = tempfile.gettempdir()
results_dir = os.path.join(temp_dir, 'results')
logs_dir = os.path.join(temp_dir, 'logs')


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


ensure_dir(results_dir)
ensure_dir(logs_dir)

print(f"Results directory exists: {os.path.exists(results_dir)}")
print(f"Logs directory exists: {os.path.exists(logs_dir)}")


training_args = TrainingArguments(
    output_dir=results_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logs_dir,
    logging_steps=10,
    save_steps=10,
    evaluation_strategy="steps"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()

model_save_path = "./results/t5_model"
tokenizer_save_path = "./results/t5_tokenizer"


os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Modelo salvo em {model_save_path}")
print(f"Tokenizador salvo em {tokenizer_save_path}")


Results directory exists: True
Logs directory exists: True


100%|██████████| 6/6 [07:31<00:00, 75.25s/it]


{'train_runtime': 451.393, 'train_samples_per_second': 0.04, 'train_steps_per_second': 0.013, 'train_loss': 34.14764912923177, 'epoch': 3.0}
Modelo salvo em ./results/t5_model
Tokenizador salvo em ./results/t5_tokenizer
52182585 Lanterna Jeep Renegade Esquerdo 52182584 Marca: Jeep, Número de peça: 52182585, Lado: Esquerdo, Tipo de veículo: Carro/Caminhonete, DescriçãoPRODUTO NOVO E ORIGINAL PRONTA ENTREGA + NOTA FISCAL + GARANTIA LANTERNA LED TRASEIRA ESQUERDA, LADO MOTORISTA- PEÇA GENUÍNA MOPAR- IMAGEM REAL DA PEÇAAPLICAÇÃO:- JEEP RENEGADE 2022 / 2023CÓDIGO :52182584VALOR DO ANÚNCIO REFERENTE A UNIDADEESTAMOS A DISPOSIÇÃO - LOUVADO SEJA DEUSGarantia do vendedor: 3 meses


In [12]:
def generate_output(input_text):

    inputs = tokenizer.encode("translate English to English: " + input_text, return_tensors="pt", max_length=512, truncation=True)


    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    output_dict = {
        "catalog_catalog.code": "",
        "catalog_catalog.full_description": "",
        "catalog_catalog.description": "",
        "catalog_catalog.similar_parts": "N/D",
        "catalog_catalog.application_table": "",
        "catalog_catalog.brand_producer": "JEEP"
    }
    
    output_values = output_text.split(",")
    if len(output_values) == 5:
        output_dict["catalog_catalog.code"] = output_values[0].strip()
        output_dict["catalog_catalog.full_description"] = output_values[1].strip()
        output_dict["catalog_catalog.description"] = output_values[2].strip()
        output_dict["catalog_catalog.application_table"] = output_values[3].strip()
        output_dict["catalog_catalog.brand_producer"] = output_values[4].strip()

    return output_dict


input_text = "735598946\tCapa Espelho Retrovisor Direito Renegade 2019\tMarca: Jeep, Número de peça: 735598946, Modelo: RENEGADE  Anos: 2021, 2020, 2019, 2018, 2017, 2016, 2015..., Tipo de veículo: Carro/Caminhonete, Lado: Direito, Material: MTR, Origem: Brasil, \tDescriçãoSe aplica em:•JEEP Renegade de 2015 a 2021Codigos:735598946Peca Genuina, com nota fiscal e garantia!Em Caso de Duvidas, Consulte Nossos Especialistas:Para evitarmos erros e possiveis devolucoes, em caso de duvidas, informe sempre o Chassi Completo do seu veiculo no campo de perguntas. Assim poderemos verificar diretamente na fabrica a correta aplicacao para o seu veiculo.Seguranca:As Pecas Originais e Genuinas sao projetadas sob medida para cada modelo de automovel, elas sao a garantia de seguranca para voce. Todos nossos produtos sao originais e vao com nota fiscal.Suporte:- Temos Equipe de Atendimento e Suporte Tecnico Especializado a sua Disposicao.Saga, casa de amigos!Marca: Aplicacao universal ou nao informada, se tiver duvida, nos faca uma pergunta.________________Codigo:735598946________________ATENCAO:- Antes de efetuar a compra tire todas suas duvidas no campo perguntas.- Mantenha seus dados e enderecos atualizados no site do Mercado Livre.- Se o Mercado Envios nao entregar seu produto,seu pagamento sera devolvido automaticamente pelo Mercado Livre.- Temos carrinho de compras, adicione varios produtos e economize no Frete.- Todos os produtos anunciados estao disponiveis em nosso estoque.- Aceitamos devolucoes apenas com a embalagem Original e Nota Fiscal. - Devido a alteracao nas politicas do Mercado Livre o frete gratis esta sujeito ao peso, preco e distancia do envio.________________SUPORTE E HORARIOS DE ATENDIMENTO:- Temos Equipe de Atendimento e Suporte Tecnico Especializado a Disposicao.- Atendimento de Seg. a Sex. das 8h00 as 18h00 e Sab. 8h00 as 12h00.________________Garantia do vendedor: 90 dias"
output = generate_output(input_text)
print(output)

{'catalog_catalog.code': '', 'catalog_catalog.full_description': '', 'catalog_catalog.description': '', 'catalog_catalog.similar_parts': 'N/D', 'catalog_catalog.application_table': '', 'catalog_catalog.brand_producer': 'JEEP'}
